## CS 372

### Assignment 2 - Decision Trees

- Grace Biggs

- Brandon L'Abbe

- Lane Thompson

- Zhiyi Zhan

In [1]:
# 
import pandas as pd

df = pd.read_excel('Personal Loan Data_DT.xlsx')
print("Dataframe: ")
print(df)

Dataframe: 
    Unnamed: 0          Unnamed: 1  \
0          NaN                 NaN   
1          NaN                 NaN   
2          NaN                 NaN   
3          NaN                 NaN   
4          NaN   Data Description:   
5          NaN                 NaN   
6          NaN                  ID   
7          NaN                 Age   
8          NaN          Experience   
9          NaN              Income   
10         NaN             ZIPCode   
11         NaN              Family   
12         NaN               CCAvg   
13         NaN           Education   
14         NaN            Mortgage   
15         NaN       Personal Loan   
16         NaN  Securities Account   
17         NaN          CD Account   
18         NaN              Online   
19         NaN          CreditCard   

                                           Unnamed: 2  
0                                                 NaN  
1                                                 NaN  
2                    